In [1]:
import dependencies
import line_profiler
import numpy as np

from Scripts.generate_data import generate_Ys
from Scripts.scBiGLasso import analyticBiGLasso
from Scripts.utilities import generate_confusion_matrices
from Scripts.utilities import precision, recall, accuracy
from Scripts.niBiGLasso import niBiGLasso

np.set_printoptions(precision=3, suppress=True)
%load_ext line_profiler
%load_ext autoreload
%autoreload 2

Python Version 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
Numpy Version 1.21.5
Scipy Version 1.7.3
Sklearn Version 1.0.2


In [2]:
# Standard way to generate matrix variate data, takes ~1 second
Psi_gen, Theta_gen, Ys = generate_Ys(
    m=100, # Note that we only need m=3 for good vindication, whereas
    p=(p:=100), # m needs to be much larger to get good estimates in general
    n=(n:=100), # anyways (in this alg and scalable version).
    expected_nonzero_psi=n**2 / 5, # (divide by 5 for sparsity)
    expected_nonzero_theta=p**2 / 5,
    structure="Kronecker Sum"
)
Ys.shape

(100, 100, 100)

In [3]:
nibig = niBiGLasso()
T_psi, T_theta = nibig.get_empiricals(Ys)
nibig.fit(T_psi, T_theta)
Psi, Theta = nibig.shrink(0.02, 0.02)
nibig.print_vindication()
print("===Psi===")
print(Psi_cm := generate_confusion_matrices(Psi, Psi_gen, mode='Negative'))
print(
    f"{precision(Psi_cm)=:.3f}",
    f"\n{recall(Psi_cm)=:.3f}",
    f"\n{accuracy(Psi_cm)=:.3f}"
)
print("\n==Theta==")
print(Theta_cm := generate_confusion_matrices(Theta, Theta_gen, mode='Negative'))
print(
    f"{precision(Theta_cm)=:.3f}",
    f"\n{recall(Theta_cm)=:.3f}",
    f"\n{accuracy(Theta_cm)=:.3f}"
)

Psi vindication: 0.9999994913096508
Theta vindication: 0.9999994795795147
===Psi===
[[ 526.  162.]
 [ 484. 8728.]]
precision(Psi_cm)=0.765 
recall(Psi_cm)=0.521 
accuracy(Psi_cm)=0.935

==Theta==
[[ 530.  182.]
 [ 510. 8678.]]
precision(Theta_cm)=0.744 
recall(Theta_cm)=0.510 
accuracy(Theta_cm)=0.930


In [12]:
# Investigate runtime
from Scripts.niBiGLasso import no_hassle
nibig = niBiGLasso()
kwargs = {
    "Ys": Ys,
    "beta_1": 0.02,
    "beta_2": 0.02,
    "vindicate": False, # Takes 90% of the runtime if true!
    "nibig": nibig
}
%lprun -f no_hassle -f nibig.get_empiricals -f nibig.fit no_hassle(**kwargs)

Typical output of the runtime calculation:
```
Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
   205                                           def no_hassle(Ys, beta_1, beta_2, vindicate=True, nibig=None):
   206         1          3.0      3.0      0.0      if nibig is None:
   207                                                   nibig = niBiGLasso()
   208         1      73866.0  73866.0     67.0      T_psi, T_theta = nibig.get_empiricals(Ys)
   209         1      10723.0  10723.0      9.7      nibig.fit(T_psi, T_theta)
   210         1      25699.0  25699.0     23.3      Psi, Theta = nibig.shrink(0.02, 0.02)
   211         1          1.0      1.0      0.0      if vindicate:
   212                                                   vinds = nibig.vindicate()
   213                                               else:
   214         1          0.0      0.0      0.0          vinds = None
   215         1          1.0      1.0      0.0      return Psi, Theta, vinds
```